# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [1]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [2]:
# Your code here:

print("=== SECTION 1: INITIAL INVENTORY ASSESSMENT ===")
print("Columns & dtypes:")
print(df.dtypes)
print("\nSample rows:")
print(df.head(10))

=== SECTION 1: INITIAL INVENTORY ASSESSMENT ===
Columns & dtypes:
[Float64, String, String, String, String, String, String, Int64]

Sample rows:
shape: (10, 8)
┌──────────────┬───────────┬───────┬─────────┬────────┬──────────┬────────┬───────┐
│ Carat Weight ┆ Cut       ┆ Color ┆ Clarity ┆ Polish ┆ Symmetry ┆ Report ┆ Price │
│ ---          ┆ ---       ┆ ---   ┆ ---     ┆ ---    ┆ ---      ┆ ---    ┆ ---   │
│ f64          ┆ str       ┆ str   ┆ str     ┆ str    ┆ str      ┆ str    ┆ i64   │
╞══════════════╪═══════════╪═══════╪═════════╪════════╪══════════╪════════╪═══════╡
│ 1.1          ┆ Ideal     ┆ H     ┆ SI1     ┆ VG     ┆ EX       ┆ GIA    ┆ 5169  │
│ 0.83         ┆ Ideal     ┆ H     ┆ VS1     ┆ ID     ┆ ID       ┆ AGSL   ┆ 3470  │
│ 0.85         ┆ Ideal     ┆ H     ┆ SI1     ┆ EX     ┆ EX       ┆ GIA    ┆ 3183  │
│ 0.91         ┆ Ideal     ┆ E     ┆ SI1     ┆ VG     ┆ VG       ┆ GIA    ┆ 4370  │
│ 0.83         ┆ Ideal     ┆ G     ┆ SI1     ┆ EX     ┆ EX       ┆ GIA    ┆ 3171  │


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [5]:
import polars as pl

try:
    df  # noqa: F821
except NameError:
    df = pl.read_csv("https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv")

df = df.rename({c: c.lower() for c in df.columns})

descriptions = {
    "carat":  "Weight of the diamond in carats (1 carat = 0.2 grams).",
    "cut":    "Cut quality grade (e.g., Fair, Good, Very Good, Premium, Ideal).",
    "color":  "Color grade (typically D = colorless through J = light color).",
    "clarity":"Clarity grade (e.g., I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF).",
    "depth":  "Total depth percentage: (z / mean(x, y)) × 100.",
    "table":  "Table width as % of average diameter.",
    "price":  "Price in U.S. dollars.",
    "x":      "Length (mm).",
    "y":      "Width (mm).",
    "z":      "Depth/height (mm).",
}

schema = list(zip(df.columns, df.dtypes))

print("=== Q2: Characteristics tracked for each diamond ===\n")
for name, dtype in schema:
    dtype_str = str(dtype)
    desc = descriptions.get(name, "(no description available)")
    print(f"- {name} [{dtype_str}]: {desc}")


=== Q2: Characteristics tracked for each diamond ===

- carat weight [Float64]: (no description available)
- cut [String]: Cut quality grade (e.g., Fair, Good, Very Good, Premium, Ideal).
- color [String]: Color grade (typically D = colorless through J = light color).
- clarity [String]: Clarity grade (e.g., I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF).
- polish [String]: (no description available)
- symmetry [String]: (no description available)
- report [String]: (no description available)
- price [Int64]: Price in U.S. dollars.


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms:

**Your answer:** Each row is a single diamond in inventory with its characteristics (carat, cut, color, clarity, dimensions) and price.


## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [10]:
# Q4: Total carats and weight of inventory
import polars as pl

# Load dataset if not already in memory
try:
    df
except NameError:
    df = pl.read_csv("https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv")
    df = df.rename({c: c.lower() for c in df.columns})

# Calculate totals
total_carat = df.select(pl.col("carat weight").sum().alias("total_carats"))

# Convert to grams and kilograms (1 carat = 0.2 grams)
weights = total_carat.with_columns(
    (pl.col("total_carats") * 0.2).alias("total_grams"),
    (pl.col("total_carats") * 0.2 / 1000).alias("total_kilograms")
)

print("=== Q4: Inventory Weight ===")
print(weights)

=== Q4: Inventory Weight ===
shape: (1, 3)
┌──────────────┬─────────────┬─────────────────┐
│ total_carats ┆ total_grams ┆ total_kilograms │
│ ---          ┆ ---         ┆ ---             │
│ f64          ┆ f64         ┆ f64             │
╞══════════════╪═════════════╪═════════════════╡
│ 8007.12      ┆ 1601.424    ┆ 1.601424        │
└──────────────┴─────────────┴─────────────────┘


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [12]:
# Q5: Variety of cuts
print("=== Q5: Variety of Cuts ===")
unique_cuts = df.select(pl.col("cut").unique())
print("Unique cut grades in inventory:")
print(unique_cuts)

=== Q5: Variety of Cuts ===
Unique cut grades in inventory:
shape: (5, 1)
┌─────────────────┐
│ cut             │
│ ---             │
│ str             │
╞═════════════════╡
│ Good            │
│ Signature-Ideal │
│ Very Good       │
│ Fair            │
│ Ideal           │
└─────────────────┘


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [14]:
# Q6: Most and least valuable diamond
print("=== Q6: Price Exposure ===")

max_price = df.select(pl.col("price").max().alias("max_price"))
min_price = df.select(pl.col("price").min().alias("min_price"))

print("Most valuable diamond (max price):")
print(max_price)
print("\nLeast valuable diamond (min price):")
print(min_price)

=== Q6: Price Exposure ===
Most valuable diamond (max price):
shape: (1, 1)
┌───────────┐
│ max_price │
│ ---       │
│ i64       │
╞═══════════╡
│ 101561    │
└───────────┘

Least valuable diamond (min price):
shape: (1, 1)
┌───────────┐
│ min_price │
│ ---       │
│ i64       │
╞═══════════╡
│ 2184      │
└───────────┘


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [15]:
# Q7: Typical prices across color grades
print("=== Q7: Pricing Patterns by Color ===")

# Calculate the average price for each color grade
avg_price_by_color = df.group_by("color").agg(pl.col("price").mean().alias("average_price")).sort("color")

print("Typical prices across color grades:")
print(avg_price_by_color)

=== Q7: Pricing Patterns by Color ===
Typical prices across color grades:
shape: (6, 2)
┌───────┬───────────────┐
│ color ┆ average_price │
│ ---   ┆ ---           │
│ str   ┆ f64           │
╞═══════╪═══════════════╡
│ D     ┆ 15255.783661  │
│ E     ┆ 11539.190231  │
│ F     ┆ 12712.241856  │
│ G     ┆ 12520.050633  │
│ H     ┆ 10487.347544  │
│ I     ┆ 8989.636364   │
└───────┴───────────────┘


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

**Your answer:**  Instead of seeing individual diamond-level details, I’m looking at summarized, category-level trends. I can now compare how different colors perform in terms of price, inventory count, or other metrics, without being distracted by row-to-row variation.

## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [16]:
# Q8: Breakdown by clarity grade
print("=== Q8: Breakdown by Clarity Grade ===")

# Group by clarity and calculate count, average price, and total carats
clarity_breakdown = df.group_by("clarity").agg([
    pl.count().alias("number_of_diamonds"),
    pl.col("price").mean().alias("typical_price"),
    pl.col("carat weight").sum().alias("total_carats")
]).sort("clarity") # Sorting by clarity for better readability

print(clarity_breakdown)

=== Q8: Breakdown by Clarity Grade ===
shape: (7, 4)
┌─────────┬────────────────────┬───────────────┬──────────────┐
│ clarity ┆ number_of_diamonds ┆ typical_price ┆ total_carats │
│ ---     ┆ ---                ┆ ---           ┆ ---          │
│ str     ┆ u32                ┆ f64           ┆ f64          │
╞═════════╪════════════════════╪═══════════════╪══════════════╡
│ FL      ┆ 4                  ┆ 63776.0       ┆ 7.87         │
│ IF      ┆ 219                ┆ 22105.844749  ┆ 316.0        │
│ SI1     ┆ 2059               ┆ 8018.864012   ┆ 2563.69      │
│ VS1     ┆ 1192               ┆ 13694.113255  ┆ 1682.74      │
│ VS2     ┆ 1575               ┆ 11809.053333  ┆ 2170.81      │
│ VVS1    ┆ 285                ┆ 16845.680702  ┆ 389.43       │
│ VVS2    ┆ 666                ┆ 14142.177177  ┆ 876.58       │
└─────────┴────────────────────┴───────────────┴──────────────┘


/tmp/ipython-input-3825966281.py:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("number_of_diamonds"),


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [17]:
# Q9: Pricing structure across cut-color combinations
print("=== Q9: Pricing Structure by Cut and Color ===")

# Group by cut and color and calculate the average price
pricing_by_cut_color = df.group_by(["cut", "color"]).agg(
    pl.col("price").mean().alias("average_price")
).sort(["cut", "color"]) # Sorting for better readability

print(pricing_by_cut_color)

=== Q9: Pricing Structure by Cut and Color ===
shape: (30, 3)
┌───────────┬───────┬───────────────┐
│ cut       ┆ color ┆ average_price │
│ ---       ┆ ---   ┆ ---           │
│ str       ┆ str   ┆ f64           │
╞═══════════╪═══════╪═══════════════╡
│ Fair      ┆ D     ┆ 6058.25       │
│ Fair      ┆ E     ┆ 5370.625      │
│ Fair      ┆ F     ┆ 6063.625      │
│ Fair      ┆ G     ┆ 7345.52381    │
│ Fair      ┆ H     ┆ 5908.5        │
│ …         ┆ …     ┆ …             │
│ Very Good ┆ E     ┆ 12101.910217  │
│ Very Good ┆ F     ┆ 12413.905495  │
│ Very Good ┆ G     ┆ 12354.013841  │
│ Very Good ┆ H     ┆ 10056.106132  │
│ Very Good ┆ I     ┆ 8930.031332   │
└───────────┴───────┴───────────────┘


**Reflection:** How specific is your analysis now? What level of detail does each row represent?

**Your answer:** New grain = each row is now filtered by cut, then color, then finding the average price for each grouping

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [18]:
# Q10: Top 5 most valuable inventory segments by color-cut combination
print("=== Q10: Top 5 Most Valuable Inventory Segments ===")

# Group by color and cut, calculate total value and number of diamonds
inventory_value_by_cut_color = df.group_by(["color", "cut"]).agg([
    pl.sum("price").alias("total_value"),
    pl.len().alias("number_of_diamonds")
])

# Sort by total value in descending order and take the top 5
top_5_segments = inventory_value_by_cut_color.sort("total_value", descending=True).head(5)

print(top_5_segments)

=== Q10: Top 5 Most Valuable Inventory Segments ===
shape: (5, 4)
┌───────┬───────────┬─────────────┬────────────────────┐
│ color ┆ cut       ┆ total_value ┆ number_of_diamonds │
│ ---   ┆ ---       ┆ ---         ┆ ---                │
│ str   ┆ str       ┆ i64         ┆ u32                │
╞═══════╪═══════════╪═════════════╪════════════════════╡
│ G     ┆ Ideal     ┆ 9363514     ┆ 690                │
│ G     ┆ Very Good ┆ 7140620     ┆ 578                │
│ F     ┆ Very Good ┆ 5648327     ┆ 455                │
│ F     ┆ Ideal     ┆ 5346782     ┆ 363                │
│ H     ┆ Ideal     ┆ 5279687     ┆ 458                │
└───────┴───────────┴─────────────┴────────────────────┘


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use